## Exercise: bank churn 
Use the bank_churn dataset to build and use a model to predict churn of bank customers.

Columns in the dataset:

![](images/bankchurn.png)

### Perform following tasks
* Determine and drop features that seem irrelevant
* Handle missing data
* Use one-hot-encoding for categorical features
* Choose prediction algorithm
* Determine the label
* Build model
* Determine accuracy of model
* Determine relative importance of features and interpret results
* Write a function to determine the probability a (unseen) customer will leave


In [2]:
# import the datafile 
import pandas as pd
url = 'https://raw.githubusercontent.com/HOGENT-Databases/BI-BigData/master/data/bank_churn.csv'
bank = pd.read_csv(url,sep=',')
bank.shape

(10000, 14)